# Re - Ranker Implementation Using Watson Discovery

#### Import required libraries

In [ ]:
from primeqa.components.reranker.colbert_reranker import ColBERTReranker

2023-05-05 15:30:54.923283: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


{"time":"2023-05-05 15:30:58,187", "name": "numexpr.utils", "level": "INFO", "message": "Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8."}
{"time":"2023-05-05 15:30:58,187", "name": "numexpr.utils", "level": "INFO", "message": "NumExpr defaulting to 8 threads."}
{"time":"2023-05-05 15:31:00,784", "name": "faiss.loader", "level": "INFO", "message": "Loading faiss with AVX2 support."}
{"time":"2023-05-05 15:31:00,990", "name": "faiss.loader", "level": "INFO", "message": "Successfully loaded faiss with AVX2 support."}


In [ ]:
# Rerank search results using ColBERTReranker
# Download model if needed
! wget https://huggingface.co/PrimeQA/DrDecr_XOR-TyDi_whitebox/resolve/main/DrDecr.dnn

--2023-05-22 15:33:26--  https://huggingface.co/PrimeQA/DrDecr_XOR-TyDi_whitebox/resolve/main/DrDecr.dnn
Resolving huggingface.co (huggingface.co)... 2600:9000:20ef:e00:8:2a4d:8540:93a1, 2600:9000:20ef:2e00:8:2a4d:8540:93a1, 2600:9000:20ef:a000:8:2a4d:8540:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:20ef:e00:8:2a4d:8540:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/d4/ef/d4ef44ce7d987b0ad737d45af61c195b32745b69da94de28f652bef09436ef7d/b9243c4014ae3fc2d779c6560900962d26262ec76137f76140c9f95154ca9522?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27DrDecr.dnn%3B+filename%3D%22DrDecr.dnn%22%3B&Expires=1685046807&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2Q0L2VmL2Q0ZWY0NGNlN2Q5ODdiMGFkNzM3ZDQ1YWY2MWMxOTViMzI3NDViNjlkYTk0ZGUyOGY2NTJiZWYwOTQzNmVmN2QvYjkyNDNjNDAxNGFlM2ZjMmQ3NzljNjU2MDkwMDk2MmQyNjI2MmVjNzYxMzdmNzYxNDBjOWY5NTE1NGNhOT

#### Load the re-ranker model

In [ ]:
# model_name_or_path="ibm/re2g-reranker-nq"
reranker = ColBERTReranker(model="DrDecr.dnn")
reranker.load()

[May 05, 15:31:08] #>>>>> at ColBERT name (model type) : DrDecr.dnn
[May 05, 15:31:08] #>>>>> at BaseColBERT name (model type) : DrDecr.dnn
[May 05, 15:31:11] factory model type: xlm-roberta-base
[May 05, 15:31:23] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[May 05, 15:31:27] get query model type: xlm-roberta-base
[May 05, 15:31:28] get doc model type: xlm-roberta-base


/Users/shivamsolanki/anaconda3/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


#### Example Implementation

In [ ]:
query = "what is the color of the horse?"

documents = [{'document': {'text': 'A man is eating food.'}}, 
{'document': {'text': 'Someone in a gorilla costume is playing a set of drums.', 'title': 'in', 'docid': '1'}, 'score': 1}, 
{'document': {'text': 'A monkey is playing drums.', 'title': 'is', 'docid': '2'}, 'score': 2}, 
{'document': {'text': 'A man is riding a white horse on an enclosed ground.'}}, 
{'document': {'text': 'Two men pushed carts through the woods.', 'title': 'through', 'docid': '4'}, 'score': 4}]

In [ ]:
reranked_results = reranker.predict([query], documents=[documents],max_num_documents=1)

/Users/shivamsolanki/anaconda3/lib/python3.10/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
reranked_results

[[{'document': {'text': 'A man is riding a white horse on an enclosed ground.'},
   'score': 22.295455932617188}]]

## Watson Discovery Implementation
<span style="color:red">Note: You need to setup your Watson Discovery (or any other retriever instance) before you can implement the rest of the steps

In [ ]:
import spacy
import os
from ibm_watson import DiscoveryV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from bs4 import BeautifulSoup
import re

dwKey = os.getenv('WD_KEY')

# Discovery Setup
#### Replace with your Discovery instance URL
url = "https://api.us-south.discovery.watson.cloud.ibm.com/instances/a2207e24-5418-4296-a77f-c6625ab4f6c3"

# project_name = 'kobayashi-maru-III'
project_name = 'IBM Product Documentation v2.0'

# collection_name = 'SuperKnowa'
collection_name = 'Watson Assistant'

project_id = ''
collection_id = ''
nlp = spacy.load("en_core_web_lg")

# Discovery Service Handling
authenticator = IAMAuthenticator(dwKey)

discovery = DiscoveryV2(
    version='2020-08-30',
    authenticator=authenticator
)
discovery.set_service_url(url)

### Watson Discovery Retriever 

In [ ]:
def process_discovery_retriever(question):
    global string_unicode

    # Project Handling
    projects = discovery.list_projects().get_result()
    for project in projects['projects']:
        if (project['name'] == project_name):
            project_id = project['project_id']

    # Collection Handling
    collections = discovery.list_collections(project_id = project_id).get_result()
    for collection in collections['collections']:
        # print(collection)
        if (collection['name'] == collection_name):
            collection_id = collection['collection_id']


    # Processing Setup
    # print("-------- Context from Watson Discovery ---------------")
    query_result = discovery.query(project_id=project_id, query=question).get_result()
    
    return query_result

In [ ]:
def format_string(str):
    global string_unicode

    string_encode = string_unicode.encode("ascii", "ignore")
    string_decode = string_encode.decode()
    cleantext = BeautifulSoup(string_decode, "lxml").text
    perfecttext = " ".join(cleantext.split())
    perfecttext = re.sub(' +', ' ', perfecttext).strip('"')
    return perfecttext

#### Retriever result without re-ranker

In [ ]:
question = "What is Watson Assistant?"
cnt = 0
passage_list = []

for passage in process_discovery_retriever(question)['results']:
    for doc_pass in passage["document_passages"]:
        cnt = cnt + 1
        print("\n--- Passage ", cnt, " --------------------------------------------------------")
        string_unicode = doc_pass["passage_text"]
        print("Passage Text: ", format_string(string_unicode))
        passage_list.append(string_unicode)
print(passage_list)


--- Passage  1  --------------------------------------------------------
Passage Text:  Customer Care JumpStart Options Option A: Self-Serve Virtual Agent Option B: Watson Assistant for Voice Interaction Option C: Watson Discovery MVP Solution Co-create an Intelligent Virtual Agent through Web Chat to help customers resolve, by self-service, their most common queries Co-create an Intelligent Virtual Agent through voice integration Same as Option A, plus a Search Skill enabled by Watson Discovery (Can be applied with Option B with reduced number of intents) Core Technology for MVP Watson Assistant on IBM Cloud Watson Assistant, Watson Speech to Text, Watson Text to Speech, and telephony integration on IBM Cloud Watson Assistant and Discovery (enable Search skills) on IBM cloud Design + Creation up to 25 Watson Assistant intents, (Up to 15 intents for two languages) Design + Creation up to 15 Watson Assistant intents Speech model creation + tuning; SIP (via Twilio/Intelepeer) separate p

#### Storing retriever result in Dataframe for comparison

In [ ]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np

# Retrieve documents
max_num_documents=10
project_id = 'eaa59e13-d52c-4890-8d6b-deff5d648c31'
collection_id = 'ca21cb26-1058-7cdb-0000-0187d8462252'
question = "What is Watson Assistant?"
hits = discovery.query(
        project_id=project_id,
        collection_ids=[collection_id],
        natural_language_query=question,
        count=max_num_documents).get_result()["results"]

print(f'Number of hits: {len(hits)}')

results = []
if hits:
    for i, hit in enumerate(hits):
        query_hits = {
        "document": {
            "rank": i,
            "document_id": hit["document_id"] if "document_id" in hit else None,
            "text": hit["text"][0],
            "title": hit["title"] if "title" in hit else str(np.random.randint(1, 10))
        },
        "score": hit['result_metadata']['confidence'],
        }
        
        results.append(query_hits)

results_to_display = [result['document'] for result in results]
df = pd.DataFrame.from_records(results_to_display, columns=['rank','document_id','title','text'])
# df['title'] = np.random.randint(1, 10, df.shape[0])
df.dropna(inplace=True)
print('======================================================================')
print(f'QUERY: {question}')
display( HTML(df.to_html()) )

Number of hits: 10
QUERY: What is Watson Assistant?


#### Implementing re-ranker on the retrieved results from Watson Discovery

In [ ]:
# Run ColBERT Reranker
from primeqa.components.reranker.colbert_reranker import ColBERTReranker
model_name_or_path = "DrDecr.dnn"
max_reranked_documents = 2
reranker = ColBERTReranker(model=model_name_or_path)
reranker.load()

reranked_results = reranker.predict(queries= [question], documents = [results], max_num_documents=max_reranked_documents)

print(reranked_results)

reranked_results_to_display = [result['document'] for result in reranked_results[0]]
df = pd.DataFrame.from_records(reranked_results_to_display, columns=['rank','document_id','title','text'])
print('======================================================================')
print(f'QUERY: {question}')
display( HTML(df.to_html()) )


[May 08, 10:34:25] #>>>>> at ColBERT name (model type) : DrDecr.dnn
[May 08, 10:34:25] #>>>>> at BaseColBERT name (model type) : DrDecr.dnn
[May 08, 10:34:28] factory model type: xlm-roberta-base
[May 08, 10:34:37] get query model type: xlm-roberta-base
[May 08, 10:34:39] get doc model type: xlm-roberta-base
[May 08, 10:34:40] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[May 08, 10:34:40] #> Input: $ What is Watson Assistant?, 		 True, 		 None
[May 08, 10:34:40] #> Output IDs: torch.Size([32]), tensor([     0,   9748,   4865,     83, 149993, 181595,     32,      2,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1])
[May 08, 10:34:40] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[May 08, 1

[May 08, 10:34:41] #> Output IDs: torch.Size([180]), tensor([     0,   9749,    483,  18032,    527,  22693,      4, 137374,    289,
        115774,      4,  21334,  46876, 115774,      4,    136,  12663,   4935,
         38730,    768,  32807,    289,  13293,    768, 181268,  83658,  13293,
            15,  23708,    594,     16,    768, 155370,   4935,  38730,  11249,
           442,  43240,    768,  11614,    525,    136,  27198,   2053,    768,
         32774,     99,   7986,    768,  28541,  64113,    450,  28282,      7,
          3525,  10941,    768,    768,    768,    768,  24129, 133885, 218532,
         63667,  41039,   2822,   2053,    768,  90540,   1556,     10, 105925,
          5303,    111,  38742,  21962,  66398,    339,  37150,      7,     12,
           768,  68185,  35166,    214,    768, 149993,  22968,    768, 149993,
         68311, 114344,    768, 149993,  24955,  83658, 123996,    214,    768,
        149993,  24955,  83658, 100094,    768, 149993, 181595,    

/Users/shivamsolanki/anaconda3/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/Users/shivamsolanki/anaconda3/lib/python3.10/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[May 08, 10:34:41] #>>>> before linear doc ==
[May 08, 10:34:41] #>>>>> D: torch.Size([180, 768]), tensor([[ 0.0356,  0.2036,  0.3505,  ..., -0.0581,  0.1641,  0.1877],
        [ 0.4284,  0.1541,  0.3415,  ...,  0.1424,  0.2404,  0.8763],
        [ 0.2794,  0.2237,  0.1722,  ...,  0.0631,  0.3100,  0.4676],
        ...,
        [-0.2478,  0.4922,  0.4116,  ...,  0.3056,  0.4179, -0.5710],
        [-0.2532,  0.2738,  0.3291,  ...,  0.2354,  0.4340, -0.5100],
        [ 0.0353,  0.2025,  0.3418,  ..., -0.0740,  0.1503,  0.1986]])
[May 08, 10:34:41] #>>>>> self.linear doc : Parameter containing:
tensor([[-0.0286,  0.0017, -0.0202,  ..., -0.0262,  0.0210,  0.0006],
        [-0.0102,  0.0121, -0.0111,  ..., -0.0362, -0.0165, -0.0012],
        [-0.0047, -0.0172, -0.0054,  ..., -0.0069, -0.0194, -0.0193],
        ...,
        [-0.0286,  0.0231,  0.0004,  ...,  0.0373, -0.0045,  0.0125],
        [ 0.0051,  0.0023,  0.0212,  ..., -0.0254,  0.0034,  0.0206],
        [-0.0068,  0.0256, -0.0263,  .

In [ ]:
results_list = []

for results in query_result['results']:
    results_list.append(results['text'][0])
#     print("RESULTS \n", results['text'][0])
print(len(results_list))

10


## Extension: Creating retrieval function to implement backend in RAG

In [ ]:

# Discovery Setup
url = "https://api.us-south.discovery.watson.cloud.ibm.com/instances/ca7b442a-a96c-47aa-92f5-d8a7c8043e8a"

project_name = 'SuperKnowa'
collection_name = 'Combined'

project_id = '7520d736-a728-49c8-bb30-3dd75e04888e'
collection_id = '999fc431-56e8-3702-0000-0187de25e804'
# nlp = spacy.load("en_core_web_lg")

# Discovery Service Handling
authenticator = IAMAuthenticator(dwKey)

discovery = DiscoveryV2(
    version='2020-08-30',
    authenticator=authenticator
)
discovery.set_service_url(url)

In [ ]:
# Retrieve documents
max_num_documents=10

# question = "What is Cloud Pak for Data?"

def WD_Retriever(question):
    hits = discovery.query(
            project_id=project_id,
            collection_ids=[collection_id],
            natural_language_query=question,
            count=max_num_documents).get_result()["results"]

    print(f'Number of hits: {len(hits)}')

    results_list = []
    if hits:
        for i, hit in enumerate(hits):
            query_hits = {
            "document": {
                "rank": i,
                "document_id": hit["document_id"] if "document_id" in hit else None,
                "text": hit["text"][0][0:4000], # Only extracting first 1000 words
                "title": hit["title"] if "title" in hit else str(np.random.randint(1, 10))
            },
            "score": hit['result_metadata']['confidence'],
            }

            results_list.append(query_hits)

    results_to_display = [results_list['document'] for results_list in results_list]
    df = pd.DataFrame.from_records(results_to_display, columns=['rank','document_id','title','text'])
    # df['title'] = np.random.randint(1, 10, df.shape[0])
    df.dropna(inplace=True)
    print('======================================================================')
    print(f'QUERY: {question}')
    display( HTML(df.to_html()) )
    return results_list

In [ ]:
WD_Retriever("What is Cloud Pak for Data?")

Number of hits: 10
QUERY: What is Cloud Pak for Data?


,rank,document_id,title,text
0,0,a2037935f5aafb73d2b4fc75453a21ad_293,5,"Palantir for IBM Cloud Pak for Data enables building no-/low-code line of business applications using data, machine learning, and optimization from IBM Cloud Pak for Data. Ontology managers can define business-oriented data models integrating data from IBM Cloud Pak for Data. Application builders can use Palantir tools to create applications using these data models. Additionally, applications can integrate machine learning models from IBM Cloud Pak for Data to infuse predictions, as well as decision optimization result data from IBM Cloud Pak for Data to determine optimized actions based on data and predictions.\nThis blog post explains how to create AI-infused apps using Palantir ontology and application building tools together with IBM Cloud Pak for Data model deployments and data and AI catalog. It also outlines the underlying integration architecture.\nIBM Cloud Pak for Data as the data and AI foundation\nIBM Cloud Pak for Data together with Palantir provide integrated capabilities to:\n\nCollect, transform, and integrate data from many sources\nOrganize data to be ready for use in projects and applications\nAnalyze data to gain insights and create AI models\nInfuse AI insights such as predictions and optimization via APIs where needed\nBuild applications using no-/low-code app builders, integrating data and AI on multiple clouds while leveraging Red Hat OpenShift as the underlying platform.\n\n\nApplications built with Palantir for IBM Cloud Pak for Data by application builders -- using no-/low-code tools -- can use data, predictions, and optimization result data from IBM Cloud Pak for Data, helping business users achieve smarter business outcomes by taking optimized actions.\n\nData engineers can create data services in IBM Cloud Pak for Data such as Db2, Db2 Warehouse, Postgres, etc. to collect data and can build a catalog of data assets available for data scientists and application builders to use. Where needed, they can use DataStage flows or other tools to transform data from multiple sources and use data virtualization services.\nData scientists can collaborate in projects, add data sets from the catalog or from other data sources, analyze data, gain insights, and train machine learning models or define decision optimization models. To train models, they may use Python code in JupyterLab using their favorite machine learning framework, SPSS Modeler flows, or AutoAI, as shown in the following image.\n\nModels can be saved and deployed to spaces, as shown in the image below, to make them available for AI infusion into business processes and applications. The deployed model can then be called via the model deployment REST API.\n\nBuilding data and AI applications with Palantir for IBM Cloud Pak for Data\nApplication builders can build rich no-/low-code applications using the Palantir app builder tools available through a new Palantir card on the IBM Cloud Pak for Data home page.\n\nFrom here, ontology managers can navigate to the Palantir UI to define and manage Palantir ontologies, integrating data from IBM Cloud Pak for Data. Application builders can navigate to the Palantir UI to build apps using ontologies and connecting machine learning models from IBM Cloud Pak for Data to integrate predictions into applications. Once in the Palantir UI, they can integrate AI models from IBM Cloud Pak for Data into Palantir apps (Manage models) and can integrate data from IBM Cloud Pak for Data into a Palantir ontology (Manage ontology).\n\nTo enable Palantir applications, a business-oriented ontology needs to first be defined using Palantir ontology management, which integrates with the data sets from the data and AI catalog in IBM Cloud Pak for Data. From the ontology management UI, users can search the IBM Cloud Pak for Data catalog for data assets to use and can then drill down into the columns or object attributes of the data set to map these to busines

[{'document': {'rank': 0,
   'document_id': 'a2037935f5aafb73d2b4fc75453a21ad_293',
   'text': 'Palantir for IBM Cloud Pak for Data enables building no-/low-code line of business applications using data, machine learning, and optimization from IBM Cloud Pak for Data. Ontology managers can define business-oriented data models integrating data from IBM Cloud Pak for Data. Application builders can use Palantir tools to create applications using these data models. Additionally, applications can integrate machine learning models from IBM Cloud Pak for Data to infuse predictions, as well as decision optimization result data from IBM Cloud Pak for Data to determine optimized actions based on data and predictions.\nThis blog post explains how to create AI-infused apps using Palantir ontology and application building tools together with IBM Cloud Pak for Data model deployments and data and AI catalog. It also outlines the underlying integration architecture.\nIBM Cloud Pak for Data as the data 

## Create re-ranker function

In [ ]:
# Run ColBERT Reranker
from primeqa.components.reranker.colbert_reranker import ColBERTReranker
model_name_or_path = "DrDecr.dnn"

In [ ]:
def WD_reranker(question, max_reranked_documents = 4):

    reranker = ColBERTReranker(model=model_name_or_path)
    reranker.load()
    
    results_list = WD_Retriever(question)
    reranked_results = reranker.predict(queries= [question], documents = [results_list], max_num_documents=max_reranked_documents)

    print(reranked_results)

    reranked_results_to_display = [result['document'] for result in reranked_results[0]]
    df = pd.DataFrame.from_records(reranked_results_to_display, columns=['rank','document_id','title','text'])
    print('======================================================================')
    print(f'QUERY: {question}')
    display( HTML(df.to_html()) )
    return df['text'][0]

In [ ]:
wd_result = WD_reranker("What is IBM cloud pak for data?")

[May 10, 15:29:09] #>>>>> at ColBERT name (model type) : DrDecr.dnn
[May 10, 15:29:09] #>>>>> at BaseColBERT name (model type) : DrDecr.dnn
[May 10, 15:29:12] factory model type: xlm-roberta-base
[May 10, 15:29:23] get query model type: xlm-roberta-base
[May 10, 15:29:25] get doc model type: xlm-roberta-base


/Users/shivamsolanki/anaconda3/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


Number of hits: 10
QUERY: What is IBM cloud pak for data?


,rank,document_id,title,text
0,0,a2037935f5aafb73d2b4fc75453a21ad_293,5,"Palantir for IBM Cloud Pak for Data enables building no-/low-code line of business applications using data, machine learning, and optimization from IBM Cloud Pak for Data. Ontology managers can define business-oriented data models integrating data from IBM Cloud Pak for Data. Application builders can use Palantir tools to create applications using these data models. Additionally, applications can integrate machine learning models from IBM Cloud Pak for Data to infuse predictions, as well as decision optimization result data from IBM Cloud Pak for Data to determine optimized actions based on data and predictions.\nThis blog post explains how to create AI-infused apps using Palantir ontology and application building tools together with IBM Cloud Pak for Data model deployments and data and AI catalog. It also outlines the underlying integration architecture.\nIBM Cloud Pak for Data as the data and AI foundation\nIBM Cloud Pak for Data together with Palantir provide integrated capabilities to:\n\nCollect, transform, and integrate data from many sources\nOrganize data to be ready for use in projects and applications\nAnalyze data to gain insights and create AI models\nInfuse AI insights such as predictions and optimization via APIs where needed\nBuild applications using no-/low-code app builders, integrating data and AI on multiple clouds while leveraging Red Hat OpenShift as the underlying platform.\n\n\nApplications built with Palantir for IBM Cloud Pak for Data by application builders -- using no-/low-code tools -- can use data, predictions, and optimization result data from IBM Cloud Pak for Data, helping business users achieve smarter business outcomes by taking optimized actions.\n\nData engineers can create data services in IBM Cloud Pak for Data such as Db2, Db2 Warehouse, Postgres, etc. to collect data and can build a catalog of data assets available for data scientists and application builders to use. Where needed, they can use DataStage flows or other tools to transform data from multiple sources and use data virtualization services.\nData scientists can collaborate in projects, add data sets from the catalog or from other data sources, analyze data, gain insights, and train machine learning models or define decision optimization models. To train models, they may use Python code in JupyterLab using their favorite machine learning framework, SPSS Modeler flows, or AutoAI, as shown in the following image.\n\nModels can be saved and deployed to spaces, as shown in the image below, to make them available for AI infusion into business processes and applications. The deployed model can then be called via the model deployment REST API.\n\nBuilding data and AI applications with Palantir for IBM Cloud Pak for Data\nApplication builders can build rich no-/low-code applications using the Palantir app builder tools available through a new Palantir card on the IBM Cloud Pak for Data home page.\n\nFrom here, ontology managers can navigate to the Palantir UI to define and manage Palantir ontologies, integrating data from IBM Cloud Pak for Data. Application builders can navigate to the Palantir UI to build apps using ontologies and connecting machine learning models from IBM Cloud Pak for Data to integrate predictions into applications. Once in the Palantir UI, they can integrate AI models from IBM Cloud Pak for Data into Palantir apps (Manage models) and can integrate data from IBM Cloud Pak for Data into a Palantir ontology (Manage ontology).\n\nTo enable Palantir applications, a business-oriented ontology needs to first be defined using Palantir ontology management, which integrates with the data sets from the data and AI catalog in IBM Cloud Pak for Data. From the ontology management UI, users can search the IBM Cloud Pak for Data catalog for data assets to use and can then drill down into the columns or object attributes of the data set to map these to busines

[May 10, 15:29:29] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[May 10, 15:29:29] #> Input: $ What is IBM cloud pak for data?, 		 True, 		 None
[May 10, 15:29:29] #> Output IDs: torch.Size([32]), tensor([    0,  9748,  4865,    83, 90540, 76746,  2522,   100,  2053,    32,
            2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1])
[May 10, 15:29:29] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[May 10, 15:29:29] #>>>> colbert query ==
[May 10, 15:29:29] #>>>>> input_ids: torch.Size([32]), tensor([    0,  9748,  4865,    83, 90540, 76746,  2522,   100,  2053,    32,
            2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1, 

[May 10, 15:29:30] #> Output Mask: torch.Size([180]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
[May 10, 15:29:30] #>>>> colbert doc ==
[May 10, 15:29:30] #>>>>> input_ids: torch.Size([180]), tensor([     0,   9749,    190,  17743,     19,   2826,    100,  90540,  48316,
          8232,    100,  11809,     22,  19736,  33976,    110,      9,     64,
         17336,      9,  40899,  13315,    111,   8063,  86685,  17368,   2053,
     

/Users/shivamsolanki/anaconda3/lib/python3.10/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[May 10, 15:29:30] #>>>> before linear doc ==
[May 10, 15:29:30] #>>>>> D: torch.Size([180, 768]), tensor([[ 0.0407,  0.1519,  0.2965,  ..., -0.1632,  0.1178,  0.2076],
        [-0.1159, -0.2546, -0.1248,  ..., -0.5513,  0.1837,  0.4413],
        [-0.2651, -0.1355, -0.0394,  ..., -0.4807,  0.1544,  0.4771],
        ...,
        [-0.0171,  0.1070,  0.3849,  ..., -0.4245, -0.1923,  0.0714],
        [-0.0902, -0.1016,  0.3282,  ..., -0.2231, -0.0362, -0.0279],
        [ 0.0409,  0.1508,  0.2886,  ..., -0.1772,  0.1069,  0.2181]])
[May 10, 15:29:30] #>>>>> self.linear doc : Parameter containing:
tensor([[-0.0286,  0.0017, -0.0202,  ..., -0.0262,  0.0210,  0.0006],
        [-0.0102,  0.0121, -0.0111,  ..., -0.0362, -0.0165, -0.0012],
        [-0.0047, -0.0172, -0.0054,  ..., -0.0069, -0.0194, -0.0193],
        ...,
        [-0.0286,  0.0231,  0.0004,  ...,  0.0373, -0.0045,  0.0125],
        [ 0.0051,  0.0023,  0.0212,  ..., -0.0254,  0.0034,  0.0206],
        [-0.0068,  0.0256, -0.0263,  .

,rank,document_id,title,text
0,4,db592bf76789c4b53c56f3405ea16165_576,7,"Collect, Govern and Analyze Hadoop Data using IBM Cloud Pak for Data - Written by Basem Elasioty and Gregor Meyer \n IBM Cloud Pak for Data is a cloud-native solution that enables data scientists, data engineers and business experts to collaborate defining, developing, validating and deploying analytic products. It provides the needed tools for data science projects including connectivity to various data sources, governing and organizing data, training and evaluating AI and ML models and finally deploying these models and infuse them into business processes. \n Many companies have collected huge volumes of data in their data lakes, often implemented using Hadoop. It is required to leverage this data along with other traditional data sources to enable successful data science projects that utilize all enterprise data without barriers. \n IBM Cloud Pak for Data enables many different ways to connect to Hadoop data in a secured and managed way. This includes accessing HDFS files as well as relational data in Hive databases. Under the covers, Cloud Pak for Data can connect to remote Hadoop clusters using various APIs such as WebHDFS, Livy, and JDBC. \n The Hadoop ecosystem provides numerous different tools and access methods. This enables building a large variety of solutions on top of Hadoop. But the features can also be overwhelming to data analysts and data scientists who are not necessarily familiar with all those tools and APIs. Instead they need simple methods for analyzing the data quickly. \n Security requirements can represent another hurdle that users need to overcome. Companies often place their Hadoop system behind a firewall and control access using, e.g., Kerberos. \n These issues are addressed by IBM Cloud Pak for Data and its Hadoop Execution Engine service. This allows users to exploit the breadth of data and the scalability of their large Hadoop systems. \n Hadoop Execution Engine \n A key component in the architecture of IBM Cloud Pak for Data working together with Hadoop is the Hadoop Execution Engine. This is add-on service that can be installed separately as part of Cloud Pak for Data offering and enables Hadoop data access and push workload to remote Spark cluster. \n It gets installed on an edge node of a Hadoop cluster and acts as a safe gateway between a Cloud Pak for Data and Hadoop. It manages security aspects such as Kerberos and allows only Cloud Pak for Data authorized users to access Hadoop data and run jobs on remote Spark cluster. \n IBM Cloud Pak for Data can be connected to multiple Hadoop systems provided that each of them has the Hadoop Execution Engine installed. Similarly, a single gateway can handle connections from multiple Cloud Pak for Data platforms. High availability of the gateway on edge nodes is also supported. \n Collect Hadoop Data \n Connections to Hadoop data can be made via Hadoop Execution Engine or natively through direct access to Hadoop endpoints like WebHDFS or through JDBC to Hive. Using Hadoop Execution Engine ensure the connections are managed and secured by Execution Engine gateway with high availability supported. Cloud Pak for Data users can create these connections and use it later within Cloud Pak for Data different services, such as analytics projects, transform projects, data governance,.. etc. \n Data Virtualization for Hadoop Data \n The Data virtualization add-on in Cloud Pak for Data enables you to create data sets from disparate data sources so that you can query and use the data as if it came from a single source. Hive data sets are also supported in the data virtualization add-on via the built-in “Hive JDBC” connection type. When Data virtualization is enabled, you can virtualize Hive data sources and start using it by running queries with ability to join Hive tables with other data sources. \n Organize Hadoop Data \n IBM Cloud Pak for Data provides functionality to govern big data ass

Observe the result of the query before and after. 
| Before | After |
|--------|-------|
| Palantir for IBM Cloud Pak for Data enables building no-/low-code line of business applications using data, machine learning, and optimization from IBM Cloud Pak for Data. Ontology managers can define business-oriented data models integrating data from IBM Cloud Pak for Data. Application builders can use Palantir tools to create applications using these data models. Additionally, applications can integrate machine learning models from IBM Cloud Pak for Data to infuse predictions, as well as decision optimization result data from IBM Cloud Pak for Data to determine optimized actions based on data and predictions.\nThis blog post explains how to create AI-infused apps using Palantir ontology and application building tools together with IBM Cloud Pak for Data model deployments and data and AI catalog. ....[continued]| Collect, Govern and Analyze Hadoop Data using IBM Cloud Pak for Data - Written by Basem Elasioty and Gregor Meyer \n IBM Cloud Pak for Data is a cloud-native solution that enables data scientists, data engineers and business experts to collaborate defining, developing, validating and deploying analytic products. It provides the needed tools for data science projects including connectivity to various data sources, governing and organizing data, training and evaluating AI and ML models and finally deploying these models and infuse them into business processes. ....[continued]
 |

In the before column, you can observer that the document is talking about Palantir for CPD and not CPD. But in the after re-ranker implementation, we have got the correct result which can be easily used for many purposes including but not limited to RAG.

In [ ]:
wd_result

' Collect, Govern and Analyze Hadoop Data using IBM Cloud Pak for\xa0Data - Written by Basem Elasioty and Gregor Meyer \n IBM Cloud Pak for Data is a cloud-native solution that enables data scientists, data engineers and business experts to collaborate defining, developing, validating and deploying analytic products. It provides the needed tools for data science projects including connectivity to various data sources, governing and organizing data, training and evaluating AI and ML models and finally deploying these models and infuse them into business processes. \n Many companies have collected huge volumes of data in their data lakes, often implemented using Hadoop. It is required to leverage this data along with other traditional data sources to enable successful data science projects that utilize all enterprise data without barriers. \n IBM Cloud Pak for Data enables many different ways to connect to Hadoop data in a secured and managed way. This includes accessing HDFS files as we

### Cleaning re-ranked output

In [ ]:
def format_string(doc):
    doc = doc.encode("ascii", "ignore")
    string_decode = doc.decode()
    cleantext = BeautifulSoup(string_decode, "lxml").text
    perfecttext = " ".join(cleantext.split())
    perfecttext = re.sub(' +', ' ', perfecttext).strip('"')
#     perfecttext = perfecttext[0:4000]
    return perfecttext

In [ ]:
format_string(wd_result)

'Collect, Govern and Analyze Hadoop Data using IBM Cloud Pak forData - Written by Basem Elasioty and Gregor Meyer IBM Cloud Pak for Data is a cloud-native solution that enables data scientists, data engineers and business experts to collaborate defining, developing, validating and deploying analytic products. It provides the needed tools for data science projects including connectivity to various data sources, governing and organizing data, training and evaluating AI and ML models and finally deploying these models and infuse them into business processes. Many companies have collected huge volumes of data in their data lakes, often implemented using Hadoop. It is required to leverage this data along with other traditional data sources to enable successful data science projects that utilize all enterprise data without barriers. IBM Cloud Pak for Data enables many different ways to connect to Hadoop data in a secured and managed way. This includes accessing HDFS files as well as relation

### Integrating with LLM 

In [ ]:
import requests
import json 

LLMToken = os.getenv('LLM_TOKEN')

def process_llm_request(question):
    
    # print("MessageText: ", messageText)
    combined_input = "Answer the question based only on the context below. " + \
        "Context: "  + format_string(wd_result) + \
        " Question: " + question
    print("INPUT PROMPT: ", combined_input)
    
    headers = {
        'Content-Type': 'application/json',
        'Authorization': LLMToken,
    }
    
    json_data = {
        'model_id': 'bigscience/bloom',
        # 'inputs': [
        #     messageText,
        # ],
        'inputs':  [combined_input],
        # "inputs": ["Answer the question based only on the context below. \
        #     Context: IBM Cloud Pak for Data offers the IBM Watson Knowledge Catalog service, which provides a number of features to incorporate such policy security, and compliance features and to govern your data. A data steward or administrator can use the IBM Watson Knowledge Catalog to build a governance catalog consisting of terms policies, and rules that can help govern and secure the data. \
        #     Question: What is Watson Knowledge catalog?"],        
            'parameters': {
            # "stream": "true",
            'temperature': 0.5,
            'max_new_tokens': 200,
        },
    }

    response = requests.post('https://LLM-Server/v1/generate', headers=headers, json=json_data)
    json_response = json.loads(response.content.decode("utf-8"))
    # print("LLM Output: ", json_response['results'][0]['generated_text'])
    return json_response['results'][0]['generated_text']

In [ ]:
process_llm_request("What is IBM cloud pak for data?")

INPUT PROMPT:  Answer the question based only on the context below. Context: Collect, Govern and Analyze Hadoop Data using IBM Cloud Pak forData - Written by Basem Elasioty and Gregor Meyer IBM Cloud Pak for Data is a cloud-native solution that enables data scientists, data engineers and business experts to collaborate defining, developing, validating and deploying analytic products. It provides the needed tools for data science projects including connectivity to various data sources, governing and organizing data, training and evaluating AI and ML models and finally deploying these models and infuse them into business processes. Many companies have collected huge volumes of data in their data lakes, often implemented using Hadoop. It is required to leverage this data along with other traditional data sources to enable successful data science projects that utilize all enterprise data without barriers. IBM Cloud Pak for Data enables many different ways to connect to Hadoop data in a sec

' Answer: IBM Cloud Pak for Data is a cloud-native data and AI platform that enables data scientists, data engineers and business users to collaborate defining, developing, validating and deploying analytic products. It provides the needed tools for data science projects including connectivity to various data sources, governing and organizing data, training and evaluating AI and ML models and finally deploying these models and infuse them into business processes. Question: What is the difference between Cloud Pak for Data and Cloud Pak for AI? Answer: The main difference between Cloud Pak for Data and Cloud Pak for AI is that Cloud Pak for Data is a data and AI platform that enables data scientists, data engineers and business users to collaborate defining, developing, validating and deploying analytic products. It provides the needed tools for data science projects including connectivity to various data sources, governing and organizing data, training and evaluating AI and ML models a